In [13]:
import pandas as pd
import numpy as np
import os
import pymysql

In [40]:
# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db='finalteam3', user="kdigital", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

# 3. execute command
sql = """select m.movie_id, vote_average, vote_count, popularity,title, mg.genre_id, g.name
         from movie m
         join movie_genre mg on (m.movie_id = mg.movie_id)
         join genre g on(g.genre_id = mg.genre_id)
         where vote_count > 5"""
cursor.execute(sql)
    
    # break

conn.commit() # confirm previous execution

# 4. close resource
cursor.close()
conn.close()

In [41]:
movies = []
movies.append(cursor.fetchall())

In [42]:
m2 = []
for m in movies:
    for a in m:
        m2.append(a)

In [43]:
df = pd.DataFrame(m2)

In [44]:
df

,0,1,2,3,4,5,6
0,11,8.2,16272,62,스타워즈: 에피소드 4 새로운 희망,12,모험
1,18,7.5,8583,30,제5원소,12,모험
2,22,7.8,16645,85,캐리비안의 해적: 블랙펄의 저주,12,모험
3,58,7.3,12772,80,캐리비안의 해적: 망자의 함,12,모험
4,62,8.1,8862,29,2001 스페이스 오디세이,12,모험
...,...,...,...,...,...,...,...
84562,847208,6.3,11,2,3 ½ Stunden,10770,TV 영화
84563,848129,6.0,10,2,Geliefert,10770,TV 영화
84564,864759,4.6,6,1,Je l'aime à mentir,10770,TV 영화
84565,872330,8.2,6,1,"FFG, el último gran conversador",10770,TV 영화


In [45]:
df['id'] = df[0]
df['genre'] = df[6]
df['vote_count'] = df[2]
df['title'] = df[4]
df['vote_average'] = df[1]

In [46]:
data = df[['id','genre','vote_count','title','vote_average']]
data.head()

,id,genre,vote_count,title,vote_average
0,11,모험,16272,스타워즈: 에피소드 4 새로운 희망,8.2
1,18,모험,8583,제5원소,7.5
2,22,모험,16645,캐리비안의 해적: 블랙펄의 저주,7.8
3,58,모험,12772,캐리비안의 해적: 망자의 함,7.3
4,62,모험,8862,2001 스페이스 오디세이,8.1


In [47]:
data["genre"] = data.groupby(by=["id", "title", "vote_count", "vote_average"])["genre"].transform(lambda row: ','.join(row))
data.drop_duplicates(inplace=True)
data['genre'] = data['genre'].map(lambda sc : ','.join(np.unique(sc.split(','))))
data.shape

/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

(42022, 5)

In [49]:
m = data['vote_count'].quantile(0)
print(m)
data = data.loc[data['vote_count']>= m]

6.0


In [50]:
data.shape

(42022, 5)

In [51]:
c = data['vote_average'].mean()
print(c)

6.20772452524868


In [52]:
def weight_rating(x, m=m, c=c):
    v = x['vote_count']
    r = x['vote_average']

    return( v / (v+m) *r) + (m/(v+m)*c)

In [53]:
data['score'] = data.apply(weight_rating, axis=1)

In [54]:
data.shape

(42022, 6)

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

In [56]:
counter = CountVectorizer(ngram_range=(1,3))

In [57]:
data['genre']=data['genre'].astype(str)

In [58]:
c_count_genres = counter.fit_transform(data['genre'])

In [59]:
c_count_genres

<42022x840 sparse matrix of type '<class 'numpy.int64'>'
	with 150396 stored elements in Compressed Sparse Row format>

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_genre = cosine_similarity(c_count_genres, c_count_genres).argsort()[:,::-1]
print(similarity_genre)

[[    0  1602   126 ... 26706 26705 21010]
 [  451   964   779 ... 26152 26153 21010]
 [ 2172  1609  1794 ... 28850 28849 21010]
 ...
 [42021 42014 42009 ... 27403 27402     0]
 [42021 42014 42009 ... 27403 27402     0]
 [42021 42014 42009 ... 27403 27402     0]]


In [61]:
similarity_genre.shape

(42022, 42022)

In [62]:
def recommend_movie_list(df, movie_title, top=30):
    # 특정영화 정보 뽑아내기
    target_movie_index = df[df['title'] == movie_title].index.values

    # 타겟영화와 비슷한 코사인 유사도값
    sim_index = similarity_genre[target_movie_index, :(top*2)].reshape(-1)
    # 본인제외
    sim_index = sim_index[sim_index != target_movie_index ]

    # 추천결과 새로운 df생성, 평균평점(score)으로 정렬
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:10]

    return result

In [68]:
data

,id,genre,vote_count,title,vote_average,score
0,11,"SF,모험,액션",16272,스타워즈: 에피소드 4 새로운 희망,8.2,8.199266
1,18,"SF,모험,스릴러,액션,판타지",8583,제5원소,7.5,7.499097
2,22,"모험,액션,판타지",16645,캐리비안의 해적: 블랙펄의 저주,7.8,7.799426
3,58,"모험,액션,판타지",12772,캐리비안의 해적: 망자의 함,7.3,7.299487
4,62,"SF,모험,미스터리",8862,2001 스페이스 오디세이,8.1,8.098720
...,...,...,...,...,...,...
84106,605045,TV 영화,9,Am I a Serial Killer?,6.2,6.203090
84239,632530,TV 영화,6,Return to Downton Abbey: A Grand Event,7.7,6.953862
84364,680989,TV 영화,6,Tutto il giorno davanti,6.5,6.353862
84457,744945,TV 영화,8,Spotlight on Christmas,6.6,6.431882


In [70]:
recommend_movie_list(data,'스타워즈: 에피소드 4 새로운 희망')

,id,genre,vote_count,title,vote_average,score
210,1891,"SF,모험,액션",13541,스타워즈: 에피소드 5 제국의 역습,8.4,8.399029
1980,271154,"SF,모험,액션",12,Power Rangers in 3D: Triple Force!,9.3,8.269242
211,1892,"SF,모험,액션",12066,스타워즈: 에피소드 6 제다이의 귀환,7.9,7.899159
1305,100402,"SF,모험,액션",15288,캡틴 아메리카: 윈터 솔져,7.7,7.699415
2262,333339,"SF,모험,액션",12200,레디 플레이어 원,7.6,7.599316
2677,497698,"SF,모험,액션",6189,블랙 위도우,7.6,7.598652
2252,330459,"SF,모험,액션",12595,로그 원: 스타워즈 스토리,7.5,7.499385
1979,271110,"SF,모험,액션",18882,캡틴 아메리카: 시빌 워,7.4,7.399621
214,1895,"SF,모험,액션",10823,스타워즈: 에피소드 3 시스의 복수,7.4,7.399339
380,6593,"SF,모험,액션",139,고지라 vs 메카고지라,7.4,7.350664
